# Fit PDF profile

This notebook illustrates how to fit 1D PDF profile in EasyDiffraction for X-ray experimental data and Ni sample using PDFfit2 (J. Phys, Condens. Matter **19**, 335219 (2007). doi: https://doi.org/10.1088/0953-8984/19/33/3352190).

In [ ]:
import os

from easyscience.fitting.fitter import Fitter
from easyDiffractionLib import Phases
from easyDiffractionLib.Jobs import Powder1DCW
from easyDiffractionLib.interface import InterfaceFactory as Calculator
from easyDiffractionLib.Profiles.P1D import PDFParameters
from easyDiffractionLib.Interfaces.pdffit2 import readGRData

# for plotting
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
output_notebook()
FIGURE_WIDTH = 900
FIGURE_HEIGHT = 300

## Define calculator

In [ ]:
calculator = Calculator()
calculator.switch("Pdffit2")

## Load experimental and reference data

`Ni-xray.gr` contains reduced X-ray data with Ni sample. Its structure is a a header with metadata and then an array of 4 columns: $r$, $G(r)$, and the error related to these 2 quantities.  
The reduction was done with `pdfgetx2`.

In [ ]:
data_fname = os.path.realpath('data\\Ni-xray.gr')
data = readGRData(data_fname)
cif_fname = os.path.realpath('data\\Ni.cif')
phases = Phases.from_cif_file(cif_fname)

## Define job 

In [ ]:
parameters = PDFParameters()

In [ ]:
job = Powder1DCW('Ni', parameters=parameters, phases=phases, interface=calculator)

fitter = Fitter(job, calculator.fit_func)

## Set parameters
### Global parameters

In [ ]:
parameters = job.parameters
parameters.get_parameters()

In [ ]:
parameters.qmax = 30
parameters.qdamp = 0.063043
parameters.wavelength = 0.126514
parameters.delta2 = 2.253193
parameters.delta1 = 0.0
parameters.qbroad = 0.1
# let's limit the range of qbroad
parameters.qbroad.min = 0.0
parameters.qbroad.max = 0.5
parameters.spdiameter = 0.0

Here is a short description for some of these parameters:
- Q_damp: PDF Gaussian dampening envelope due to limited $Q$-resolution. The Gaussian envelope is of the form $\exp(-\frac{(r Q_{damp})^2}{2})$
- Q_broad: PDF peak broadening from increased intensity noise at high $Q$
- delta1: coefficient for $(1/r)$ contribution to the peak sharpening.
- delta2: coefficient for $(1/r^2)$ contribution to the peak sharpening.

Using these notations, the PDF peak width is expressed as 

$$ \sigma_{ij} \sqrt{1-\frac{\delta_1}{r_{ij}}-\frac{\delta_2}{r_{ij}^2}+Q_{broad}^2 r_{ij}^2},$$ where $i$, $j$ correspond to two different atoms.  

It contains contributions from thermal and zero point displacements and static disorder. $\sigma_{ij}'$ is the peak width without correlation.   
The first two terms correct for the effects of correlated motion. The term $\delta_2/r^2$ describes the low temperature behavior, and $\delta_1/r$ describes the high temperature case. Since the two parameters are highly correlated, one will in practice choose which one to refine.  

The last term models the PDF peak broadening as a result of the Q-resolution of the diffractometer. In many cases this term will only be significant for refinements over wider r -ranges. Note that the Q resolution also results in an exponentialdampening of the PDF peaks which is modeled using the parameter Q_damp.

### Parameters for each phase

Here we only have one phase.

In [ ]:
phases

#### Print some phase settings 

In [ ]:
print(job.phases[0].cell)
print(job.phases[0].center)
print(job.phases[0].spacegroup)
print(job.phases[0].scale)

In [ ]:
job.phases[0].get_parameters()

#### Modify some of settings

In [ ]:
job.phases[0].atoms[0].adp.Uiso = 0.005445
job.phases[0].scale = 0.366013
job.phases[0].cell.length_a = 3.52

## Calculate profile

In [ ]:
x_data = data[:, 0]

y_data = job.create_simulation(x_data)

## Select parameters to optimize

In [ ]:
job.phases[0].cell.length_a.fixed = False
job.phases[0].scale.fixed = False
parameters.qdamp.fixed = False
parameters.qbroad.fixed = False
parameters.delta1.fixed = False
parameters.delta2.fixed = False


## Run the fit

In [ ]:
result = fitter.fit(x_data, data[:, 1], 
                    method='least_squares', minimizer_kwargs={'diff_step': 1e-5, 'verbose': 2})

## Print fitting results

In [ ]:
print(f"The fit has been successful: {result.success}")  
print(f"The goodness of fit (chi2) is: {result.reduced_chi}")
print("The optimized parameters are:")
for param in job.get_fit_parameters():
    print(f"{param.name}: {param.value} +\- {param.variance} {param.unit}") 

## Plot

In [ ]:
y_data = calculator.fit_func(x_data)

In [ ]:
# obtain data from PdfFit calculator object
Gobs = data[:, 1]
Gfit = y_data
Gdiff = Gobs - Gfit
Gdiff_baseline = -10

Gdiff_show = Gdiff + Gdiff_baseline

In [ ]:
fig = figure()
fig.xaxis.axis_label = 'r (Å)'
fig.yaxis.axis_label = r"$$G (Å^{-2})\$$"
fig.title.text = 'Fit of nickel to x-ray experimental PDF'

fig.scatter(x_data, Gobs, legend_label='G(r) Data', fill_alpha=0., line_color='steelblue', marker='circle')
fig.line(x_data, Gfit, legend_label='G(r) Fit', color='orangered', line_width=2)
fig.line(x_data, Gdiff_show, legend_label='G(r) Diff', color='grey', line_width=2)
show(fig)
